### Basel II RWA calculations

1. The purpose of the notenook is to perform RWA calculations as described in the https://www.bis.org/publ/bcbs128.pdf
2. The results are tested against test cases as provided by the BIS
3. The maturity flooring is a simplification (see 32.51 in https://www.bis.org/basel_framework/chapter/CRE/32.htm?inforce=20230101&published=20200327)

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Settings
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
def calculate_RWA(exposure_type, PD, LGD, EAD, add_on=1.06, M=None, S=None):
    """
    Calculate Risk-Weighted Assets (RWA) for different types of exposures.
    
    Parameters:
    - exposure_type (str): Type of regulatory exposure class.
    - PD (float): Probability of Default. Must be between 0 and 1.
    - LGD (float): Loss Given Default. Must be between 0 and 1.
    - EAD (float): Exposure at Default.
    - M (float, optional): Maturity for corporate exposures. Must be between 1 and 3. Default is None.
    - S (float, optional): Sales for SME corporate exposures. Must be between 1 and 50. Default is None.
    
    Returns:
    - float: Risk-Weighted Assets (RWA) based on the given inputs.
    
    Raises:
    - AssertionError: If PD or LGD are not between 0 and 1.
    """
    
    # Validate input ranges
    assert 0 <= PD <= 1, "PD must be between 0 and 1."
    assert 0 <= LGD <= 1, "LGD must be between 0 and 1."
    if M is not None:
        M = np.clip(M, 1, 5)
    if S is not None:
        S = np.clip(S, 5, 50)
    
    # The G function, the inverse of the standard normal cumulative distribution function
    def G(z):
        return norm.ppf(z)

    # Calculate Correlation R based on exposure type
    if exposure_type == "Residential Mortgages":
        R = 0.15
    elif exposure_type == "Qualifying Revolving Retail Exposures":
        R = 0.04
    elif exposure_type == "Other Retail Exposures":
        R = 0.03 * (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) + 0.16 * (1 - (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)))
    elif exposure_type == "Corporate Exposures":
        R = 0.12 * (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) + 0.24 * (1 - (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)))
        if S is not None:
            R -= 0.04 * (1 - (S - 5) / 45)
    else:
        return np.nan
    
    # Calculate Capital Requirement K
    if exposure_type == "Corporate Exposures":
        # Maturity adjustment b for corporate exposures
        b = (0.11852 - 0.05478 * np.log(PD)) ** 2
        K = (LGD * norm.cdf(np.sqrt((1 - R)**-1) * G(PD) + np.sqrt(R / (1 - R)) * G(0.999)) - PD * LGD) * (1 - 1.5 * b) ** -1 * (1 + (M - 2.5) * b)
    else:
        K = LGD * norm.cdf(((1 - R)**-0.5) * G(PD) + (R / (1 - R)) ** 0.5 * G(0.999)) - PD * LGD
    
    # Calculate Risk-Weighted Assets RWA
    RWA = K * 12.5 * EAD * add_on
    
    return RWA

In [3]:
# Test the function with incorrect PD and LGD values to see if assertions work
try:
    print("RWA for Corporate Exposure:", calculate_RWA("Corporate Exposures", 1.2, 0.4, 100, 3))
except AssertionError as e:
    print(e)

try:
    print("RWA for Corporate Exposure:", calculate_RWA("Corporate Exposures", 0.02, -0.4, 100, 3))
except AssertionError as e:
    print(e)

# Example usage
PD = 0.02  # Probability of Default
LGD = 0.4  # Loss Given Default
EAD = 100  # Exposure at Default
M = 3     # Maturity
S = 10    # Sales for SME
add_on = 1.06

print("RWA for Residential Mortgage:", calculate_RWA("Residential Mortgages", PD, LGD, EAD, add_on))
print("RWA for Qualifying Revolving Retail:", calculate_RWA("Qualifying Revolving Retail Exposures", PD, LGD, EAD, add_on))
print("RWA for Other Retail:", calculate_RWA("Other Retail Exposures", PD, LGD, EAD, add_on))
print("RWA for Corporate Exposure:", calculate_RWA("Corporate Exposures", PD, LGD, EAD, M, add_on))
print("RWA for SME Corporate Exposure:", calculate_RWA("Corporate Exposures", PD, LGD, EAD, M, S, add_on))

PD must be between 0 and 1.
LGD must be between 0 and 1.
RWA for Residential Mortgage: 82.85433774748495
RWA for Qualifying Revolving Retail: 27.251803169301215
RWA for Other Retail: 54.63611515913094
RWA for Corporate Exposure: 257.42410788421637
RWA for SME Corporate Exposure: 301.5092152166892


In [4]:
# Data provided in the Annex 5 of https://www.bis.org/publ/bcbs128.pdf
data = [
    [0.03, 14.44, 11.30, 4.15, 2.30, 4.45, 8.41, 0.98, 1.85],
    [0.05, 19.65, 15.39, 6.23, 3.46, 6.63, 12.52, 1.51, 2.86],
    [0.10, 29.65, 23.30, 10.69, 5.94, 11.16, 21.08, 2.71, 5.12],
    [0.25, 49.47, 39.01, 21.30, 11.83, 21.15, 39.96, 5.76, 10.88],
    [0.40, 62.72, 49.49, 29.94, 16.64, 28.42, 53.69, 8.41, 15.88],
    [0.50, 69.61, 54.91, 35.08, 19.49, 32.36, 61.13, 10.04, 18.97],
    [0.75, 82.78, 65.14, 46.46, 25.81, 40.10, 75.74, 13.80, 26.06],
    [1.00, 92.32, 72.40, 56.40, 31.33, 45.77, 86.46, 17.22, 32.53],
    [1.30, 100.95, 78.77, 67.00, 37.22, 50.80, 95.95, 21.02, 39.70],
    [1.50, 105.59, 82.11, 73.45, 40.80, 53.37, 100.81, 23.40, 44.19],
    [2.00, 114.86, 88.55, 87.94, 48.85, 57.99, 109.53, 28.92, 54.63],
    [2.50, 122.16, 93.43, 100.64, 55.91, 60.90, 115.03, 33.98, 64.18],
    [3.00, 128.44, 97.58, 111.99, 62.22, 62.79, 118.61, 38.66, 73.03],
    [4.00, 139.58, 105.04, 131.63, 73.13, 65.01, 122.80, 47.16, 89.08],
    [5.00, 149.86, 112.27, 148.22, 82.35, 66.42, 125.45, 54.75, 103.41],
    [6.00, 159.61, 119.48, 162.52, 90.29, 67.73, 127.94, 61.61, 116.37],
    [10.00, 193.09, 146.51, 204.41, 113.56, 75.54, 142.69, 83.89, 158.47],
    [15.00, 221.54, 171.91, 235.72, 130.96, 88.60, 167.36, 103.89, 196.23],
    [20.00, 238.23, 188.42, 253.12, 140.62, 100.28, 189.41, 117.99, 222.86]
]

# Dummy 3-level column names
columns = pd.MultiIndex.from_tuples([
    ('PD', '', ''),
    ('Corporate Exposures', 0.45, 50),
    ('Corporate Exposures', 0.45, 5),
    ('Residential Mortgages', 0.45, 0),
    ('Residential Mortgages', 0.25, 0),
    ('Other Retail Exposures', 0.45, 0),
    ('Other Retail Exposures', 0.85, 0),
    ('Qualifying Revolving Retail Exposures', 0.45, 0),
    ('Qualifying Revolving Retail Exposures', 0.85, 0),
], names=['Segment', 'LGD', 'SME'])

# Create the original DataFrame again
df = pd.DataFrame(data, columns=columns)

# Set 'PD' as index for stacking
df['PD'] = df[('PD', '', '')]
df_dropped = df.drop(('PD', '', ''), axis=1)
df_dropped = df_dropped.set_index(df['PD'])

# Perform the stack operation
df_stacked = df_dropped.stack(level=[0, 1, 2]).reset_index(name='Value')
df_stacked.columns = ['PD', 'Segment', 'LGD', 'SME', 'Value']

# Applying the function row-wise
bis_add_on = 1.0
df_stacked['RWA_calc'] = df_stacked.apply(lambda row: calculate_RWA(row['Segment'], 
                                                                    row['PD'] / 100, 
                                                                    row['LGD'], 
                                                                    100, 
                                                                    bis_add_on,
                                                                    2.5, 
                                                                    row['SME'],
                                                                    ), axis=1)

df_stacked['RWA_calc'] = df_stacked['RWA_calc'].round(3)
df_stacked['diff'] = abs(1- df_stacked['RWA_calc'] / df_stacked['Value'])
df_stacked.head(70)

,PD,Segment,LGD,SME,Value,RWA_calc,diff
0,0.03,Corporate Exposures,0.45,5,11.30,11.299,0.000088
1,0.03,Corporate Exposures,0.45,50,14.44,14.444,0.000277
2,0.03,Other Retail Exposures,0.45,0,4.45,4.451,0.000225
3,0.03,Other Retail Exposures,0.85,0,8.41,8.408,0.000238
4,0.03,Qualifying Revolving Retail Exposures,0.45,0,0.98,0.980,0.000000
5,0.03,Qualifying Revolving Retail Exposures,0.85,0,1.85,1.851,0.000541
6,0.03,Residential Mortgages,0.25,0,2.30,2.305,0.002174
7,0.03,Residential Mortgages,0.45,0,4.15,4.149,0.000241
8,0.05,Corporate Exposures,0.45,5,15.39,15.396,0.000390
9,0.05,Corporate Exposures,0.45,50,19.65,19.651,0.000051


In [5]:
print(f"The maximum difference between calculated and provided RW is {df_stacked['diff'].max():.2%}")

The maximum difference between calculated and provided RW is 0.22%
